In [1]:
import pandas as pd
import numpy as np
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 
from datetime import timedelta
import ast

In [2]:
accounts_attendance = pd.read_csv('./dumps/votes_csv/accounts_attendance.csv')
accounts_blockrecord = pd.read_csv('./dumps/votes_csv/accounts_blockrecord.csv')
accounts_failpaymenthistory = pd.read_csv('./dumps/votes_csv/accounts_failpaymenthistory.csv')
accounts_friendrequest = pd.read_csv('./dumps/votes_csv/accounts_friendrequest.csv')

accounts_group = pd.read_csv('./dumps/votes_csv/accounts_group.csv')
accounts_nearbyschool = pd.read_csv('./dumps/votes_csv/accounts_nearbyschool.csv')
accounts_paymenthistory = pd.read_csv('./dumps/votes_csv/accounts_paymenthistory.csv')
accounts_pointhistory = pd.read_csv('./dumps/votes_csv/accounts_pointhistory.csv')

accounts_school = pd.read_csv('./dumps/votes_csv/accounts_school.csv')
accounts_timelinereport = pd.read_csv('./dumps/votes_csv/accounts_timelinereport.csv')
accounts_user_contacts = pd.read_csv('./dumps/votes_csv/accounts_user_contacts.csv')
accounts_user = pd.read_csv('./dumps/votes_csv/accounts_user.csv')

accounts_userquestionrecord = pd.read_csv('./dumps/votes_csv/accounts_userquestionrecord.csv')
accounts_userwithdraw = pd.read_csv('./dumps/votes_csv/accounts_userwithdraw.csv')
event_receipts = pd.read_csv('./dumps/votes_csv/event_receipts.csv')
events = pd.read_csv('./dumps/votes_csv/events.csv')

polls_question = pd.read_csv('./dumps/votes_csv/polls_question.csv')
polls_questionpiece = pd.read_csv('./dumps/votes_csv/polls_questionpiece.csv')
polls_questionreport = pd.read_csv('./dumps/votes_csv/polls_questionreport.csv')
polls_questionset = pd.read_csv('./dumps/votes_csv/polls_questionset.csv')
polls_usercandidate = pd.read_csv('./dumps/votes_csv/polls_usercandidate.csv')